In [ ]:
def stock_fdr(stock_name, start_date = None, end_date = None):
    import pandas as pd
    import FinanceDataReader as fdr
    import datetime
    import os
    
    #1. 저장 파일 형식에 필요한 Date 불러오기
    today_year = datetime.datetime.today().year
    today_month = datetime.datetime.today().month
    today_day = datetime.datetime.today().day
    
    #2. KRX종목 DataFrame 존재여부 확인
    if os.path.isfile("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day)):
        krx_df = pd.read_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day), index_col=[0])
    else:
        krx_df = fdr.StockListing("KRX")
        krx_df.to_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day))
    
    #3. 종목 코드 찾기
    if stock_name in krx_df['Name'].values:
        symbol = krx_df[krx_df['Name'] == stock_name]['Symbol'].values[0]
    else:
        raise "There is no name in krx"
    
    #4. 종목 데이터 불러오기
    stock_df = fdr.DataReader(symbol, start_date, end_date)
    
    return stock_df


In [ ]:
stock_name = "롯데케미칼"
start_date = '2021'
df = stock_fdr(stock_name = stock_name, start_date = start_date)
df

In [ ]:
import plotly.graph_objects as go
candlestick = go.Candlestick(x = df.index, open = df['Open'], high = df['High'], low = df['Low'], close = df['Close'], name = 'Candle Stick')
fig = go.Figure(candlestick)
fig.update_xaxes(rangeslider_visible=False)
fig.show()

# DD/MDD

In [ ]:
def mdd_fn(df):
    df = df[['Close']].copy()
    def return_fn(df):
        return df['Close'].pct_change().fillna(0)
    def cum_return_fn(df_return):
        return (1 + df_return).cumprod()
    df['Return'] = return_fn(df)
    df['CumReturn'] = cum_return_fn(df['Return'])
    df['MaxCumReturn'] = df['CumReturn'].cummax()
    df['DrawDown'] = (df['CumReturn'] / df['MaxCumReturn']) -1
    mdd = df['DrawDown'].min()
    list_info = list()
    # 최저가
    min_close_value = df['Close'].min()
    # 최고가
    max_close_value = df['Close'].max()
    print("최고: {}\n최저: {}\nMDD: {}".format(max_close_value, min_close_value,round(mdd*100,2)))
    df_max_close = df[df['DrawDown'] == 0].copy()
    df_max_close.loc[df.index[len(df)-1]] = 0
    period = df_max_close.index[1:] - df_max_close.index[:-1]
    mdd_days = period.days
    max_period = mdd_days.max()
    max_period_idx = mdd_days.argmax()
    print("MAX Draw Down: {} ~ {}".format(df_max_close.index[:-1][max_period_idx].date(), df_max_close.index[1:][max_period_idx].date()))
    print("{} days".format(max_period))
    print("max period : {}".format(max_period))
    list_info.append(max_close_value)
    list_info.append(min_close_value)
    list_info.append(round(mdd*100,2))
    list_info.append(df_max_close.index[:-1][max_period_idx].date())
    list_info.append(df_max_close.index[1:][max_period_idx].date())
    list_info.append(max_period)
    return df, list_info

In [ ]:
df_dd, stock_info = mdd_fn(df)
df_dd
stock_info

In [ ]:
import plotly.graph_objects as go
dd = go.Scatter(x= df_dd.index, y = df_dd['DrawDown'], name = "Draw Down", fill="tozeroy")
fig = go.Figure(dd)
fig.show()

# MACD 그리기

In [ ]:
def macd_oscillator(df, short_N, long_N, signal_N):
    df = df[['Close']].copy()
    df['Short'] = df['Close'].ewm(span = short_N, adjust = False).mean()
    df['Long'] = df['Close'].ewm(span = long_N, adjust = False).mean()
    df['MACD'] = df['Short'] - df['Long']
    df['Signal'] = df['MACD'].ewm(span = signal_N, adjust = False).mean()
    df['MACD Oscillator'] = df['MACD'] - df['Signal']
    return df[['MACD', 'Signal', 'MACD Oscillator']]

In [ ]:
df_macd = macd_oscillator(df, short_N = 9, long_N = 26, signal_N = 13)
df_macd

In [ ]:
import plotly.graph_objects as go
macd = go.Scatter(x=df.index, y=df_macd["MACD"], name="MACD")
signal = go.Scatter(x=df.index, y=df_macd["Signal"], name="signal")
oscillator = go.Bar(x=df.index, y=df_macd["MACD Oscillator"], name="oscillator")
fig = go.Figure(macd)
fig.add_trace(signal)
fig.add_trace(oscillator)
fig.show()

# RSI 그리기

In [ ]:
def rsi_fn(df, rsi_period):
    import numpy as np
    df_close = df[['Close']].copy()
    df_close['Change'] = df_close['Close'] - df_close['Close'].shift(1)
    df_close['Gain'] = np.where(df_close['Change']>=0, df_close['Change'], 0)
    df_close['Loss'] = np.where(df_close['Change']<0, df_close['Change']*-1, 0)
    df_close['avgGain'] = df_close['Gain'].rolling(window = rsi_period).mean()
    df_close['avgLoss'] = df_close['Loss'].rolling(window = rsi_period).mean()
    df_close['RS'] = df_close["avgGain"] / df_close['avgLoss']
    df_close['RSI'] = 100 - (100/(1+df_close['RS']))
    return df_close

In [19]:
df_rsi = rsi_fn(df, rsi_period = 14)
df_rsi

,Close,Change,Gain,Loss,avgGain,avgLoss,RS,RSI
Date,,,,,,,,
2021-01-04,279500,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2021-01-05,282500,3000.0,3000.0,0.0,NaN,NaN,NaN,NaN
2021-01-06,283000,500.0,500.0,0.0,NaN,NaN,NaN,NaN
2021-01-07,304000,21000.0,21000.0,0.0,NaN,NaN,NaN,NaN
2021-01-08,301000,-3000.0,0.0,3000.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-07-11,173000,-7000.0,0.0,7000.0,1178.571429,2678.571429,0.440000,30.555556
2022-07-12,168000,-5000.0,0.0,5000.0,1178.571429,2750.000000,0.428571,30.000000
2022-07-13,169000,1000.0,1000.0,0.0,1250.000000,2285.714286,0.546875,35.353535


In [20]:
import plotly.graph_objects as go
rsi = rsi = go.Scatter(x=df_rsi.index, y=df_rsi['RSI'], name="RSI")
fig = go.Figure(rsi)

rsi_low_line = go.Scatter(x=df_rsi.index, y=(df_rsi['RSI']/df_rsi['RSI']).fillna(1)*30,line=dict(color='red',dash='dash'),name='Low bound')
rsi_high_line = go.Scatter(x=df_rsi.index, y=(df_rsi['RSI']/df_rsi['RSI']).fillna(1)*70,line=dict(color='red',dash='dash'),name='High bound')

fig.add_trace(rsi_low_line)
fig.add_trace(rsi_high_line)

fig.show()

In [21]:
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,278000,282500,277000,279500,342443,0.012681
2021-01-05,277500,283000,276500,282500,268642,0.010733
2021-01-06,288000,291500,282000,283000,454092,0.001770
2021-01-07,291000,305000,290000,304000,711593,0.074205
2021-01-08,306500,307000,296000,301000,449301,-0.009868
...,...,...,...,...,...,...
2022-07-11,180000,181000,172000,173000,184136,-0.038889
2022-07-12,173500,173500,167000,168000,131272,-0.028902
2022-07-13,170000,172000,168000,169000,99388,0.005952


In [22]:
volume = go.Bar(x=df.index, y=df["Volume"], name="Volume")
fig = go.Figure(volume)
fig.show()